- bigram model is a type of statistical language model that predicts the probability of a word or character based on the single preceding word or character. 

- Training Data: The model is trained on a large corpus of text. 
- Bigram Extraction: It identifies all consecutive pairs of words or characters (bigrams) within the training data. For example, in "the cat sat," the bigrams are "the cat" and "cat sat". 
- Frequency Counting: The model counts how many times each bigram appears. 
Probability Calculation: The probability of a subsequent item is calculated by dividing the count of the specific bigram by the total count of the preceding item. For instance, the probability of "cat" following "the" would be the count of "the cat" divided by the count of "the". 

In [193]:
with open ("input.txt", "r", encoding="utf-8") as f: 
    text = f.read(); 

# encoding: It translates characters to numbers
# read the file to inspect it

In [194]:
print("length of the data set: ", len(text))

length of the data set:  1115394


In [195]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [196]:
#all the unique characters in the dataset: 
# create a hashmap -> turn it into a list -> sort it -> unique
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(" ".join(chars))
print(vocab_size) # so there are 65 unique characters


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [197]:
for i,ch in enumerate(chars): 
    print(i, ch)

0 

1  
2 !
3 $
4 &
5 '
6 ,
7 -
8 .
9 3
10 :
11 ;
12 ?
13 A
14 B
15 C
16 D
17 E
18 F
19 G
20 H
21 I
22 J
23 K
24 L
25 M
26 N
27 O
28 P
29 Q
30 R
31 S
32 T
33 U
34 V
35 W
36 X
37 Y
38 Z
39 a
40 b
41 c
42 d
43 e
44 f
45 g
46 h
47 i
48 j
49 k
50 l
51 m
52 n
53 o
54 p
55 q
56 r
57 s
58 t
59 u
60 v
61 w
62 x
63 y
64 z


In [198]:
# Tokenization is the process of converting raw text into smaller units called tokens.
# In the context of language models, a token can be a character, word, or subword.
# For character-level models (like this one), each character is treated as a token.
# creating a mapping from characters to integers

# creates a dictionary called 'stoi' (string-to-integer), where each unique character in 'chars' is mapped to a unique integer index.
# For example, if chars = ['a', 'b', 'c'], then stoi = {'a': 0, 'b': 1, 'c': 2}.
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# defines an encoder function that converts a string into a list of integers,
# where each character in the string is mapped to its corresponding integer index using the 'stoi' dictionary.
# For example, encode("abc") would return [stoi['a'], stoi['b'], stoi['c']].
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode("hello")))
print(" ")
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 43, 50, 50, 53]
hello
 
[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [199]:
# using libray to encode and decode
import tiktoken
enc = tiktoken.get_encoding("gpt2") # get the encoding for gpt2 model 
tokens = enc.encode("hello")
print(tokens)
decoded = enc.decode(tokens)
print(decoded)
print(" ")
print(enc.encode("hii there"))
print(enc.decode(enc.encode("hii there")))

[31373]
hello
 
[71, 4178, 612]
hii there


In [200]:
# encode the entire text dataset and sotre it into a torch tensor
# A torch tensor is a multi-dimensional array, similar to a NumPy array, but with additional capabilities for GPU acceleration and automatic differentiation.
import torch 
data = torch.tensor(encode(text), dtype=torch.long) # should be of int64 type; 
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [201]:
# split up the data into train and validation sets
n = int(0.9*len(data)) # 90% of the data for training, rest val
train_data = data[:n]
val_data = data[n:]
print("Train data:")
print(train_data[:100])
print(" ")
print("Validation data:")
print(val_data[:100])

# we want to create a dataset class that will handle the data loading and batching

Train data:
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
 
Validation data:
tensor([12,  0,  0, 19, 30, 17, 25, 21, 27, 10,  0, 19, 53, 53, 42,  1, 51, 53,
        56, 56, 53, 61,  6,  1, 52, 43, 47, 45, 46, 40, 53, 59, 56,  1, 14, 39,
        54, 58, 47, 57, 58, 39,  8,  0,  0, 14, 13, 28, 32, 21, 31, 32, 13, 10,
         0, 19, 53, 53, 42,  1, 51, 53, 56, 56, 53, 61,  6,  1, 52, 43, 47, 45,
        46, 40, 53, 59, 56,  1, 19, 56, 43, 51, 47, 53,  8,  0, 19, 53, 42,  1,
        57, 39, 60, 43,  1, 63, 53, 59,  6,  1])


In [202]:
block_size = 8 # context length 
first_chars = train_data[:block_size+1] # this is the first 9 characters of the training data
# Convert tensor elements to Python ints before decoding
print(first_chars)
decoded_text = decode([int(idx) for idx in first_chars])
print(decoded_text)

# This is a list comprehension that iterates over each element 'idx' in the tensor 'first_chars'.
# Each 'idx' is a PyTorch tensor scalar, so 'int(idx)' converts it to a standard Python integer.
# The result is a list of Python integers corresponding to the character token IDs.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
First Cit


- all these charcters follow each other when they follow each other
- 8 individual examples pack together 
- if 18, 47 exist together, than 56 is very likely to come after and so on

In [203]:
x = train_data[:block_size+1]
y = train_data[1:block_size+2]

# y is offset by 1 from x because we want to predict the next character

# size: 3 
# x: [1,2,3,4] (size + 1)
# y: [1,2,3,4,5] (size + 2)

for t in range(block_size):
    context = x[:t+1]  # This will reach up to but not including the last character when t = block_size - 1, since x has length block_size + 1
    target = y[t]
    print(f"when input is {context} the target is {target}")
    
    
# this approach allow the model to get used to a single character to a hugh chunk from text
# useful coz in inference, we want to be able to generate a text of any length (form one char to a huge chunk of text)

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [204]:
# batch dimensions: 
# many batches of multiple chunk of texts stacked up in a single sensor so that the model can process them in parallel
# those chunks are being processed in parallel, but the model is still processing one character at a time
torch.manual_seed(1337) # this is to make the random number generator deterministic
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?


# the following function is used to generate a batch of data

def get_batch(split):
    # generate a small batch of data of inputs x and targets y

    # Select the appropriate dataset based on the split ('train' or 'val')
    data = train_data if split == 'train' else val_data

    # Randomly choose 'batch_size' starting indices for the input sequences.
    # Each index will be the start of a chunk of text of length 'block_size'.
    
    # Example: Suppose len(data) = 100 and block_size = 8, batch_size = 4
    # torch.randint(100 - 8, (4,)) might return tensor([23, 56, 12, 70])
    # These are the starting indices for each sequence in the batch.
    
    # (4,) is the shape of a 1-dimensional tensor (or numpy array) with 4 elements.
    
    # Why do we need to subtract 8? We subtract block_size (8) from the data length (100) to ensure that each randomly chosen starting index leaves enough room for a full sequence of length block_size. If we didn't subtract 8, some starting indices would go past the end of the data when we try to extract a chunk of length 8.
    # so its shape is (4,). The comma indicates it's a tuple, and the single value means 1D.
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    # so this will return 4 random integers between 0 and 92 (100 - 8)

    # For each random index, extract a sequence of length 'block_size' as input (x)
    # and the next sequence (shifted by 1) as the target (y).
    # This way, for each input sequence, the model learns to predict the next character at every position.
    
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])

    # Example of how it works:
    # Suppose:
    #   - len(data) = 100
    #   - block_size = 8
    #   - batch_size = 4
    # Why is this (4,) but the stack below is (8,)? 
    # random_indices = tensor([23, 56, 12, 70])  # Shape: (4,) -- one start index per batch
    # When we stack slices of length block_size (which is 8), each slice is (8,), so stacking 4 of them gives (4, 8)
    # For each index:
    # Example:
    #   - x[0] = data[23:31], y[0] = data[24:32]
    #   - x[1] = data[56:64], y[1] = data[57:65]
    #   - x[2] = data[12:20], y[2] = data[13:21]
    #   - x[3] = data[70:78], y[3] = data[71:79]
    
    
    # [23:31] means a slice of the data tensor, starting at index 23 (inclusive) and ending at index 31 (exclusive).
    # In Python, data[23:31] returns the elements at positions 23, 24, ..., 30 (but not 31).
    # used to extract a sequence of length 8 (since 31 - 23 = 8) from the data.
    # In other words, for each batch, x is a chunk of 8 items, and y is the same chunk but shifted by 1.
    # So, x is a (4, 8) tensor of input sequences, and y is a (4, 8) tensor of target sequences (the next character for each position).

    return x, y

![Explaining torch.stack method](images/Explaining-torch.stack-method-2048x1430.jpg)

In [205]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("")
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context} the target is {target}")
# [57, 43, 60, 43, 52,  1, 63, 43] this is one of the batch of inputs
# [43, 60, 43, 52,  1, 63, 43, 58] this is the corresponding batch of targets -> that will be used to calculate the loss after being passed to the transformer model

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target is 1
w

In [206]:
print(xb)
# get four stacks of input, each stack of (8,1)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [207]:
# bigram language model: 
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # for the reproducibility of the results


class BigramLanguageModel(nn.Module): # subclass of nn.Module
    def __init__(self, vocab_size): 
        
        super().__init__() # call the constructor of the parent class (nn.Module)
        
        # creates an embedding table where each token in the vocabulary is mapped directly to a vector of size vocab_size.
        # In this setup, the embedding table acts as a simple lookup table: for each input token, it outputs a vector (logits) that represents the scores for all possible next tokens.
        # This means that the model is essentially learning, for each token, the probability distribution over the next token, without considering any context beyond the current token.
        # thin wrapper around the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx is a tensor of shape (batch_size, block_size)
        logits = self.token_embedding_table(idx) # (batch_size, time, channel)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            # stretching out the array, and add C as the second dimension: 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # cross entropy loss -> measure the quality of the model's predictions with respect to the true targets (how well are we predicting the next character)
            #only the target hsould have a reallly high dimension, the rest should be 0 or close to zero
            
        return logits, loss
    # logits is the score for the next character in the sequence 
    # predicting what come next based on the current character 
    
    
    # The following code defines a method to generate new tokens (for example, text) from the model, given an initial sequence of token indices.
    # - The method takes in `idx`, which is a tensor containing the current context (shape: batch size B, sequence length T).
    # - For a specified number of steps (`max_new_tokens`), it repeatedly:
    #   1. Feeds the current sequence to the model to get predictions (logits).
    #   2. Focuses on the logits(scores) for the last token in the sequence, which represent the model's prediction for the next token.
    #   3. Applies a softmax to convert logits into probabilities.
    #   4. Samples the next token index from this probability distribution.
    #   5. Appends the sampled token to the sequence, extending the context.
    # - After generating the desired number of new tokens, it returns the full sequence including the newly generated tokens.

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # idx is the current context, shape (B,T)
            logits, loss = self(idx)
            
            # get the logits for the last time step
            logits = logits[:, -1, :] # (B,C): get predictions for the last time step
            # last element in the time dimension and then stretch it out to (B,C)
            
            # convert logits to probabilities
            probs = F.softmax(logits, dim=-1) # convert logits to probabilities
            
            # sample next token
            idx_next = torch.multinomial(probs, num_samples=1) # sample next token
            
            # append to sequence
            idx = torch.cat((idx, idx_next), dim=-1) # append to sequence (b, t+1)
        return idx

In [208]:
m = BigramLanguageModel(vocab_size)
logit, loss = m(xb, yb)
print(logit.shape)
print(loss)   


# we are expecting a loss around -ln(1/65) = 4.17
# that is using the uniform distribution to predict the next character
# a value higher than that implies that the model is not diffuse and has a bit of entropy, such that it failed to learn the distribution

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [209]:
idx = torch.zeros((1,1), dtype=torch.long) # B, T = 1,1 holding zeros
# 0 is standing as the new line character -> feed in as the first character

# generates 100 new tokens from the model, starting with an initial context of a single token (the newline character, represented by 0).
# The model's generate method returns a tensor of token indices, which is then converted to a list.
# The decode function translates these token indices back into readable text, which is printed out.
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) 

# the code generated doesnt make sense because: 
# assuming we want to predict the character at position t, we only need the character at position t-1
# but we feed in the whole context of the text so far, and use the last character as the input
# history is not used rn 


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [210]:
"""
Token embedding is a technique used in natural language processing to convert discrete tokens (such as words or characters) into continuous vector representations. Each unique token in the vocabulary is assigned a vector of real numbers, typically of fixed size. These vectors are learned during the training process and capture semantic or syntactic properties of the tokens.
In the context of neural networks, especially transformer-based models, token embeddings serve as the initial input to the model. Instead of processing raw token indices, the model works with their corresponding embeddings, which allows it to learn relationships and patterns in the data more effectively.
For example, in PyTorch, `nn.Embedding(vocab_size, embedding_dim)` creates a lookup table where each token index maps to an embedding vector of size `embedding_dim`. In the BigramLanguageModel above, the embedding table is set up so that each token is mapped directly to a vector of size equal to the vocabulary size, which is then used to predict the next token.
"""

'\nToken embedding is a technique used in natural language processing to convert discrete tokens (such as words or characters) into continuous vector representations. Each unique token in the vocabulary is assigned a vector of real numbers, typically of fixed size. These vectors are learned during the training process and capture semantic or syntactic properties of the tokens.\nIn the context of neural networks, especially transformer-based models, token embeddings serve as the initial input to the model. Instead of processing raw token indices, the model works with their corresponding embeddings, which allows it to learn relationships and patterns in the data more effectively.\nFor example, in PyTorch, `nn.Embedding(vocab_size, embedding_dim)` creates a lookup table where each token index maps to an embedding vector of size `embedding_dim`. In the BigramLanguageModel above, the embedding table is set up so that each token is mapped directly to a vector of size equal to the vocabulary 

In [211]:
# create a pytorch optimization object
# simpler model torch.optim.SGD(m.parameters(), lr=1e-3)
# typical learnign rate: 3e-4, 1e-3, 3e-2
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [212]:
# This section of code trains the language model using mini-batch gradient descent.
# It sets the batch size, then runs a training loop for 10,000 steps.
# In each step:
#   - It samples a batch of training data (input and target sequences).
#   - It computes the model's predictions and the loss.
#   - It clears previous gradients, performs backpropagation to compute new gradients, and updates the model's parameters using the optimizer.
# After training, it prints the final loss value.

batch_size = 32 # how many independent sequences will we process in parallel?
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    
    # backpropagate the loss
    # zero_grad means that the gradients are set to zero, and set_to_none=True means that the gradients are not stored
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    
    # update the weights
    optimizer.step()
print(loss.item())

2.382369041442871


In [213]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))
# doing the same thing as before but now the model is trained, if we sample from it again, it will generate coherent text (not best pereformance, but a lot better)
# the model is not communicating with the rest of the text


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


The mathematical trick in self-attention:` we want different information talk to each other in the sentence. `
Self-attention is a mechanism that allows each position in a sequence to attend to all positions in the same sequen ce, including itself. It's a key component of transformer architectures that enables the model to capture long-range dependencies and relationships between different parts of the input.

The core idea is that for each position in the sequence, the model computes attention weights that determine how much focus to place on each other position when processing that particular position. This is done through three learned linear transformations:

1. **Query (Q)**: Represents "what am I looking for?"
2. **Key (K)**: Represents "what do I have to offer?"  
3. **Value (V)**: Represents "the actual content/information"

The attention mechanism computes a weighted sum of values, where the weights are determined by the compatibility between queries and keys. The mathematical formula is:

**Attention(Q,K,V) = softmax(QK^T/√d_k)V**

Where:
- QK^T computes the similarity between queries and keys
- The scaling factor √d_k prevents the dot products from becoming too large
- softmax normalizes the attention weights to sum to 1
- The final result is a weighted combination of the values

This allows the model to dynamically focus on different parts of the input sequence when processing each position, making it particularly effective for tasks involving sequential data like language modeling.

Note: 
We do not want the nth token to talk to n+1 token or the future tokens, because they are irrelevant to the current character. 
Current context should only communicated with previous context, not later. -> `Communicate with the past`

Approach: we take the average of all the preceding elements: 
For example, if we are at the 5th position in the sequence, we want to compute attention using only the information from positions 1, 2, 3, 4, and 5 (the current position and all previous positions). We do NOT use information from positions 6, 7, 8, etc. because those are "future" tokens that haven't been generated yet during training. This creates a causal mask where each position can only attend to itself and all positions that came before it in the sequence.

- Creates a feature vector that summarizes me in the context of my histroy. 
- Yet this form of communication is extremely lossy, the interaction between characters are weak, and we lose a lot of information about the spatial arrangements of all those tokens. 
- We will solve that in late step. 

- For every single element independently, for every teeth token in that sequence, we would like to calculate the average of all the vectors in all the previous tokena and all previous tokens. 

In [214]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # Batch, time, channel
x = torch.randn(B,T,C) # have information at each point of the sequence
print(x.shape)
# print(x)

# this is a 4x8 tensor, each element is either 0 or 1


torch.Size([4, 8, 2])


Example of a (B, T, C) tensor where:
 - **B = 3** (batch size - 3 different sequences)
 - **T = 5** (time steps - 5 tokens in each sequence) 
 - **C = 4** (channels/features - 4-dimensional embedding for each token)
 
 ```python
 torch.manual_seed(42)  # for reproducible results
 example_tensor = torch.randn(3, 5, 4)
 print("Shape:", example_tensor.shape)
 print("Tensor:")
 print(example_tensor)
 ```
 
 **Interpretation:**
 - **Batch 0** (first sequence): `example_tensor[0]` → 5×4 matrix (5 tokens, each with 4 features)
 - **Batch 1** (second sequence): `example_tensor[1]` → 5×4 matrix (5 tokens, each with 4 features)  
 - **Batch 2** (third sequence): `example_tensor[2]` → 5×4 matrix (5 tokens, each with 4 features)
 
 Each element `example_tensor[b, t, c]` represents:
 - **b**: which sequence in the batch (0, 1, or 2)
 - **t**: which position in the sequence (0, 1, 2, 3, or 4)
 - **c**: which feature dimension (0, 1, 2, or 3)
 
 **Example:** `example_tensor[1, 3, 2]` is the 3rd feature of the 4th token in the 2nd sequence



In [215]:
# version 1 using basic algebra

# we want x[b,t] = mean_{i<=t} x[b,i]
# xbow = x bag of words when we are avergaeing up things
# Create a tensor to store the bag-of-words representation
# Each position will contain the average of all previous tokens (including current)
xbow = torch.zeros((B,T,C)) 

# For each batch element
for b in range(B):
    # For each time step in the sequence
    for t in range(T):
        # Calculate the mean of all tokens from start up to and including current position
        # x[b,:t+1] gets all tokens from index 0 to t (inclusive)
        # .mean(dim=0) averages across the time dimension, giving us a C-dimensional vector
        xprev = x[b,:t+1]   #  (t,c) all the teeth tokens, with c as the 2D information from little tokens
        xbow[b,t] = torch.mean(xprev, dim=0) # dimension = 0 means average across the time dimension (first axis)
      

print(xbow.shape)
print(xbow)

# we hv established the correct form of xbow 


torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [216]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [217]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

The first tensor shows the original token embeddings for batch 0:
Each row represents a token at position t, with 2 feature dimensions
- Position 0: [0.1808, -0.0700] - first token's embedding
- Position 1: [-0.3596, -0.9152] - second token's embedding  
- Position 2: [0.6258, 0.0255] - third token's embedding
- And so on for all 8 tokens in the sequence

The second tensor shows the bag-of-words (averaged) representation:
Each row now contains the running average of all previous tokens (including current)
- Position 0: [0.1808, -0.0700] - same as original (only 1 token to average)
- Position 1: [-0.0894, -0.4926] - average of tokens 0 and 1
- Position 2: [0.1490, -0.3199] - average of tokens 0, 1, and 2
- Position 3: [0.3504, -0.2238] - average of tokens 0, 1, 2, and 3
- And so on...

Notice how the values become more "smoothed" as we average more tokens,
creating a cumulative representation that captures the context up to each position,
but the idea is too inefficient when it comes to performing a huge set of data.

In [218]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
print(a)
print(b)



tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])


In [219]:
# we want the lower part to be all ones 
lower_triangular_mask = torch.tril(torch.ones(3,3))

# we also want the upper part to be all ones 
upper_triangular_mask = torch.triu(torch.ones(3,3))

print(lower_triangular_mask)
print(upper_triangular_mask)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])


In [220]:
# This demonstrates how to compute cumulative sums using matrix multiplication
# The lower triangular matrix acts as a "mask" that sums all previous elements
torch.manual_seed(42)
d = torch.tril(torch.ones(3,3))  # Lower triangular matrix of ones
e = torch.randint(0,10,(3,2)).float()  # Random data matrix
f = d @ e  # Matrix multiplication computes cumulative sums
print(f)

tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [221]:
# to get the mean: 
d = d/ torch.sum(d, dim=1, keepdim=True)
# dim = 1 means we're summing along dimension 1 (columns), so for each row we sum all the values in that row
# This creates a column vector where each element is the sum of the corresponding row
# keepdim=True preserves the original shape, so we get a 3x1 matrix instead of a 1D tensor
print(d)
f = d @ e 
print(f)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [222]:
# then we can do a matrix multiplication, note that the . means dot product 
c = a @ b

print(c)

tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [223]:
# version 2, using tril 

weight = torch.tril(torch.ones(T,T))
wei = weight / weight.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [224]:
xbow2 = wei @ x # this is the bag of words, but we want to do it in a way that the information cannot flow from the future to the past 
# This matrix multiplication performs weighted aggregation across the time dimension
# (B,T,T) represents attention weights where each row sums to 1 (normalized)
# (B,T,C) represents the input embeddings for each token at each time step
# The result (B,T,C) gives each token a weighted combination of all previous tokens
# 
# For each batch B and time step T, we compute:
# output[b,t,c] = sum(weight[b,t,k] * input[b,k,c] for k in range(t+1))
# This ensures information only flows from past to present (causal attention)
torch.allclose(xbow, xbow2) # check if they are the same

True

In [225]:
xbow[0], 
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [226]:
# version 3 using softmax 
triangular_mask = torch.tril(torch.ones(T,T))
weight_softmax = torch.zeros((T,T)) # in future, they are going to be data dependent and not zero
# some tokens may be more interested in the future tokens rather than the past tokens, depending on the values, they are going to find different affinities of the data
weight_softmax = weight_softmax.masked_fill(triangular_mask == 0, float('-inf'))
# setting them to -inf means that the softmax will be 0 for those positions, they are not allowed to talk to each other/ previous tokens
weight_softmax = F.softmax(weight_softmax, dim=-1) # normalization across the row
weight_softmax 

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [227]:
xbow3 = weight_softmax @ x
torch.allclose(xbow, xbow3)


True

In [228]:
xbow[0], xbow3[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

Self-Attention Mechanism: Query, Key, and Value Vectors

- In self-attention, each token embedding produces three distinct vectors:

- Query Vector (Q): Represents "what am I looking for?" - this vector encodes
the information that the current token is seeking from other positions in the sequence.

- Key Vector (K): Represents "what do I contain?" - this vector encodes the 
information that each token has available to share with other tokens.

- Value Vector (V): Contains the actual information/content that gets passed
to other tokens based on the attention weights.

- Attention Score Calculation:
To determine how much attention each token should pay to others, we compute
the affinity between queries and keys using dot product similarity:
Attention_Score = Q · K^T

- The dot product measures alignment between vectors - when a query vector
is well-aligned with a key vector (pointing in similar directions), their
dot product will be large, indicating high affinity. Conversely, misaligned
vectors will have small or negative dot products.

learn data-dependent attention patterns where
tokens can selectively focus on the most relevant information from other
positions in the sequence, rather than treating all previous tokens equally.

> A single head refers to one attention mechanism that learns a specific pattern of how tokens should attend to each other. Each head can focus on different types of relationships (e.g., syntactic, semantic, positional) and creates its own unique attention weights. Multiple heads allow the model to capture diverse attention patterns simultaneously.



- Bias is an additional learnable parameter in neural network layers that allows the model to shift the activation function. It's essentially a constant value that gets added to the weighted sum of inputs before applying the activation function.
```
output = activation(weighted_sum + bias)
where weighted_sum = input1 * weight1 + input2 * weight2 + ... + inputN * weightN
```

Why bias is important:
- **Flexibility**: Without bias, the activation function can only pass through the origin (0,0). Bias allows the function to be shifted horizontally, giving the model more flexibility to fit the data.
- **Learning capability**: It enables the model to learn patterns that don't pass through the origin, which is crucial for most real-world problems.
- **Decision boundary**: In classification tasks, bias helps position the decision boundary optimally in the feature space.

In the context of the linear layer `nn.Linear(C, head_size, bias=False)`:
- Setting `bias=False` means the layer will NOT learn a bias term
- This is sometimes done to reduce parameters or when you want the layer to be purely linear without any offset
- For attention mechanisms, bias is often omitted because the attention weights are normalized anyway

Example:

In [239]:
# version 4 self attention block 
from math import inf
# Set random seed for reproducible results
torch.manual_seed(1337)

# Define dimensions: Batch size=4, Time steps=8, Channels=32
B,T,C = 4,8,32 
# Create random input tensor representing token embeddings
x = torch.randn(B,T,C)

# Create lower triangular matrix (1s below diagonal, 0s above)
# This ensures causal attention - tokens can only attend to previous positions
tril = torch.tril(torch.ones(T,T))

# Initialize attention weights as zeros, but later we will modify the weighting, we do not want uniform weighting, since some of the chracters might be of higher importance, we want the thing we are looking for to flow to us. 
# we want to gather information from the past but we want to do it in the data dependent way. 
# single head self attention 


# no communicarion is happening here, we are just projecting the data into a smaller space, we are not attending to anything yet 

head_size = 16 # the number of dimensions in the query, key, and value vectors
key = nn.Linear(C, head_size, bias = False) 
query = nn.Linear(C, head_size, bias = False) 
k = key(x)   # (B,T,head_size)
q = query(x) # (B,T,head_size)

# communication is happening here, dot product:
wei = q @ k.transpose(-2, -1) # (B,T,T) -> why transpose? because we want to do a dot product between the query and the key, and we want to do it in the time dimension, not the channel dimension
# we want (B,T,16) @ (B,16,T) -> (B,T,T)
tril = torch.tril(torch.ones(T,T))


out = wei @ x

# Output shape is (B,T,C) - same as input but with attention applied
out.shape

torch.Size([4, 8, 32])

In [241]:
wei[0] # raw affinities between all the node

tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

In [ ]:

# wei = torch.zeros((T,T))


wei = wei.masked_fill(tril == 0, float(-inf)) # not allowed to communicated
# Apply softmax to normalize attention weights across each row
# Each row now sums to 1, representing attention distribution
wei = F.softmax(wei, dim = -1)
# dim = -1 because we want to apply softmax across the last dimension (the time dimension T)
# This ensures that for each query position, the attention weights sum to 1 across all key positions
# Each row in the attention matrix represents attention distribution for one query position
# Apply attention weights to input embeddings
# This creates weighted combinations where each token aggregates information
# from all previous tokens (including itself) based on attention scores

out = wei @ x

In [ ]:
wei[0]
# data dependent matter and how much of data is useful to aggregrate from any of these tokens

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

 
- When a key vector has a high value in a specific channel (dimension), and a query vector also has a high value in the same channel, their dot product will result in a high affinity score
- This high affinity indicates that these two positions share similar patterns or features in that particular dimension
- Through the softmax normalization, this high affinity gets amplified and converted into a large attention weight
- This allows the current position to learn and incorporate information from the previous position that has similar characteristics
- For example, if we're at position 8 looking for consonants, and position 3 has a key vector with high values in the consonant-related channels, their dot product will be large
- After softmax, position 8 will pay strong attention to position 3, effectively learning consonant patterns from the previous context

In [ ]:
wei 
# The attention weights are now data-dependent, considering both the position and content of tokens
# Each row represents how much attention each position pays to all previous positions (including itself)
# The weights are learned through the query-key dot product and normalized via softmax 

# e.g. i am in wei[0][0][0][8], i am at position 8 and contains the vowel, i am looking for some constants from 0 - 7



tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [238]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)